# Monads Revisited

Let's go back to monads. In Haskell, a monad is a type class defined as
```haskell
class Functor m => Monad m where
    (<=<) :: (b -> m c) -> (a -> m b) -> (a -> m c)
    return :: a -> m a
```

In [1]:
using Pkg
Pkg.activate(".")
using MLStyle

include("aux_functors.jl");

  Activating project at `~/MEGA/EMAP/Julia_Tutorials/CategoryTheory`


## Example - List Monad

In [1]:
struct Nil end
struct Cons{T}
  val::T
  next::Union{Cons{T},Nil} # n.b. abstract type!
end
List{T} = Union{Cons{T}, Nil}
fmap(f::Function, x::Nil) = Nil()
fmap(f::Function, x::Cons) = Cons(f(x.val),fmap(f, x.next))

fmap (generic function with 2 methods)

In [2]:
concatenate(x::Nil ,y::List) = y
concatenate(x::Cons,y::List) = Cons(x.val, concatenate(x.next, y))

μ(x::Nil) = Nil()
μ(x::Cons{<:List})  = concatenate(x.val, μ(x.next))
η(x,::Type{List}) = Cons(x,Nil())
bind(x::List, f::Function) = μ(fmap(f, x))


bind (generic function with 1 method)

In [3]:
concatplus1(x::Int)::List  = Cons(x+1,Cons(x, Nil()))
x = Cons(1,Nil())

μ(fmap(concatplus1,bind(x, concatplus1)))

Cons{Int64}(3, Cons{Int64}(2, Cons{Int64}(2, Cons{Int64}(1, Nil()))))

In [4]:
μ(Cons(x,Cons(x,Cons(x,Nil()))))

Cons{Int64}(1, Cons{Int64}(1, Cons{Int64}(1, Nil())))

In [5]:
Cons(10,Cons(10,Nil()))

Cons{Int64}(10, Cons{Int64}(10, Nil()))

## Example - Maybe Monad 

Remeber that `Maybe` is an endofunctor. If we have a natural transformation 
$\eta$ and $\mu$ such that $(\text{Maybe}, \eta, \mu)$ is a monoid, then we have our monad.

In [6]:
η(x,::Type{Maybe})   = Just(x)
μ(x::Just{<:Just})   = Just(x._1._1)
μ(x::Just{<:Nothing})= Nothing()
μ(x::Nothing)        = Nothing()

μ(Just(η(10, Maybe)))
bind(mx::Maybe, f::Function) = μ(fmap(f, mx))
⊙(f::Function,g::Function) = x -> bind(g(x),f)

safesqrt(x::Real) = x < 0 ? Nothing() : Just(√x)
safesolve(x::Real) = x == 0 ? Nothing() : Just(10 / x)
@show (safesolve ⊙ safesqrt)(14)
@show (safesolve ⊙ safesqrt)(-10)
@show (safesolve ⊙ safesqrt)(0);

(safesolve ⊙ safesqrt)(14) = Just{Float64}(2.6726124191242437)
(safesolve ⊙ safesqrt)(-10) = Nothing()
(safesolve ⊙ safesqrt)(0) = Nothing()


In [7]:
struct unit{T} end
unit{Maybe}(x) = η(x,Maybe)

## Example - State Monad

<!-- The State functor 
 State (\s -> let (a, s') = g s in (f a, s')) -->

In [8]:
# struct State
#     s::String
# end

# struct ST{a}
#     "State -> (a, State)"
#     f::Function 
# end

# app(s::ST) =
# # ST(x::Int->x)
# f(x::Int) = x+1

In [9]:
# struct State{T}
#     "s->(a,s)"
#     f::T
# end
struct State
    "s->(a,s)"
    f::Function
end
function (s::State)(state = nothing)
  s.f(state)
end

fmap(f::Function, g::State) = State(s-> begin
        a,s_new = g(s)
        f(a), s_new
    end)

s1 = State(s -> (0, s))
s1_update = fmap(x->x+2,s1)

s1_update("ok")

(2, "ok")

In [10]:
η(x) = State(s->(x,s))

bind(st::State, f::Function) = State(s -> begin
        x,s_new = st(s)
        f(x)(s_new)
    end)

↣ = bind

bind (generic function with 3 methods)

In [11]:
myf(x::Int) = State(s->(string(x),s*"<\\svg>"))
bind(s1, myf)("<SVG>")

("0", "<SVG><\\svg>")

In [12]:
(s1 ↣ myf)("<SVG>")

("0", "<SVG><\\svg>")

### Using State Monads - Relabelling trees

We want to define a function that takes a `Tree a` and returns a `Tree Int`, where each
value in the nodes is exchanged for an integer. For example:
```
Tree{String}
    *
   / \
  *   c
 / \
a   b

----

Tree{Int}
    *
   / \
  *   2
 / \
0   1
```

In [13]:
@data Tree{a} begin
    Leaf(::a)
    Node(::Tree{a}, ::Tree{a})
end

In [14]:
t = Node(Node(Leaf("a"),Leaf("b")),Leaf("c"))

Node{String}(Node{String}(Leaf{String}("a"), Leaf{String}("b")), Leaf{String}("c"))

In [15]:
fresh = State(s->(s,s+1))
mlabel(lx::Leaf) = fresh ↣ n -> η(Leaf(n))
mlabel(n::Node) = begin
    mlabel(n._1) ↣ (l -> mlabel(n._2) ↣ r-> η(Node(l, r)))
end;

In [16]:
label(t::Tree)::Tree{Int} = first(mlabel(t)(0))
label(t)

Node{Int64}(Node{Int64}(Leaf{Int64}(0), Leaf{Int64}(1)), Leaf{Int64}(2))